In [3]:

from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120, 
    return_messages=True # return the messages as message class, not just the text
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human." ),
    MessagesPlaceholder(variable_name="history"), #Put unknown messages from unknown sources here
    ("human", "{question}")
])

#history: default memory key

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm 

def invoke_chain(question):
    result = chain.invoke({
    "question": question
    })
    memory.save_context({"input": question}, {"output": result.content})
    print(result)

invoke_chain("My name is nico")



content='Nice to meet you, Nico! How can I assist you today?'


In [4]:
invoke_chain("What is my name?")

content='Your name is Nico.'
